# Load Data

In [ ]:
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import os
import matplotlib.pyplot as plt
import math
import numpy as np
from lib import read_investments,drawdown,plot_drawdown,cagr

investments ={
    "Bitcoin":"BTC-USD",    
    "Tesla":"TSLA",    
    "S&P500":"VFINX",
    "Bond":"VBMFX",
    "Gold": "GC=F",
}

investments_dataframe= read_investments()
print(investments_dataframe.head(10))
print(investments_dataframe.tail(10))

In [ ]:


sd=datetime(2015,1,1)
ed=datetime(2019,12,31)
truncated_df = investments_dataframe.fillna(method='ffill').truncate(before=sd,after=ed)
# Get MDD
print("MDD")
for name in investments.keys():
    print(drawdown(truncated_df[name]).min())
print("CAGR")
# Get CAGR
period=720
for name in investments.keys():
    start =(truncated_df[name].iloc[0])
    end = (truncated_df[name].iloc[-1])
    cagrs = truncated_df[name].rolling(period).apply(func=lambda v:cagr(v.iloc[0],v.iloc[-1],period),raw=False)
    total_cagr = cagr(start,end, 365*4)
    print(cagrs.std())

In [ ]:

drawdown_period =30

for name in investments.keys():
    print(name)
    s=investments_dataframe[name].fillna(method='ffill')
    drawdowns = s.rolling(drawdown_period).apply(func=drawdown,raw=False)
    mdd = drawdowns.min()
    x=list(map(lambda v:drawdowns.quantile(math.exp(-v*0.1)),np.arange(0,50)))
    print(drawdowns.quantile(0.01))
    y = list(map(lambda v:math.log((drawdowns<=v).sum()/drawdowns.count()),x))
    plt.scatter (x,y,label=name)
    plt.legend()
plt.title(f'Drawdown in {period} days')
plt.xlabel('Drawdown')
plt.ylabel('log(P(x))')

In [ ]:
check_period=720
q_values=(0.1,0.03,0.01)
drawdown_periods=(5,10,15)
df =pd.DataFrame(index=investments.keys())
for name in investments.keys():
    print(name)
    #propagate last valid observation forward
    s=investments_dataframe[name].fillna(method='ffill')
    print(s.count())
    for drawdown_period in drawdown_periods:
        print (f'Drawdown period {drawdown_period}')
        drawdowns = s.rolling(drawdown_period).apply(func=drawdown,raw=False)
        for q in q_values:
            quantiles = drawdowns.rolling(check_period).apply(lambda v:v.quantile(q),raw=False)
            rel_std = quantiles.std()/quantiles.mean()
            print(f'Q:{q}, releatvie std:{rel_std}')
            df.at[name,f'DD_{drawdown_period}_Q_{q}']=rel_std
print("-----")
print(df)
df.to_csv(path_or_buf='drawdown.csv')